In [ ]:
## STEP ONE - Installing dependencies if necessary

#!pip install pytz
#!pip install pylast
#!pip install xlrd
#!pip install pandas
#!pip install psycopg2
#!pip install spotipy
#!pip install googletrans
#!pip install sklearn

In [ ]:
## STEP TWO - Import dependencies

import datetime as dt
import calendar
from dateutil.tz import gettz
import time
import pylast
from IPython.display import HTML
import ipywidgets as widgets
import base64
import pandas as pd
from pandas import Timestamp
import csv
from datetime import date
import time
import getpass
import psycopg2 as ps 
import socket
import ssl #bypasses SSL certificate fail issue for pylast
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from googletrans import Translator
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [ ]:
## STEP THREE - Load DatabaseConnection class for PostgreSQL integration
#             - Enter database connection details in _init_

class DatabaseConnection:
    #Initialize class
    def __init__(self):
        db_name =''
        db_user = ''
        db_pass= ''
        db_host='localhost'
        db_port=5432
        try:
            self.connection = ps.connect("dbname="+db_name+" user="+db_user+" password="+db_pass+" host="+db_host+" port="+str(db_port))
            self.connection.autocommit = True
            self.cursor = self.connection.cursor()
        except:
            print("Unable to connect to database.")

    # For Step 6
    def create_table(self):
        create_table_command = "CREATE TABLE lastFM (id serial PRIMARY KEY, artist varchar(500) NOT NULL, track varchar(500) NOT NULL, album varchar(500) NOT NULL, timestamp integer NOT NULL)"
        self.cursor.execute(create_table_command)
        
    def insert_new_record(self, artist, track, album, timestamp):
        insert_command = "INSERT INTO lastFM(artist, track, album, timestamp) VALUES('"+ artist+"','"+track+"','"+album+"','"+timestamp+"')"
        self.cursor.execute(insert_command)

    # For Step 7
    def iterate(self):
        self.cursor.execute("SELECT * FROM lastFM")
        return self.cursor.fetchall()
    
    def remove_dup(self, listid):
        rem_cmd = "DELETE FROM lastFM WHERE id="+str(listid)
        self.cursor.execute(rem_cmd)
        
    # For Step 8
    def audio_feature_table(self):
        make = "CREATE TABLE audio_features(artist varchar(500) NOT NULL, track varchar(500) NOT NULL, t_played integer NOT NULL, id serial PRIMARY KEY, spot_id varchar(30), danceability float, energy float, key integer, loudness float, mode integer, speechiness float, acousticness float, instrumentalness float, liveness float, valence float, tempo float, t_sig integer);"
        self.cursor.execute(make)
        transfer = "INSERT INTO audio_features SELECT artist, track, COUNT(*) FROM lastFM GROUP BY artist, track HAVING COUNT(*)>=1;"
        self.cursor.execute(transfer)
    
    # For Step 9
    def iter_features(self):
        self.cursor.execute("SELECT * FROM audio_features")
        return self.cursor.fetchall()
    
    def add_spotID(self, t_id, spotID):
        add_cmd = "UPDATE audio_features SET spot_id = '"+str(spotID)+"' WHERE id= "+str(t_id)
        self.cursor.execute(add_cmd)
    
    # For Step 10
    def rem_spot_null(self):
        self.cursor.execute("DELETE FROM audio_features WHERE spot_id IS NULL")
        
    # For Step 11
    def feature_track(self, t_id, dance, energy, key, loudness, mode, speech, acoustic, instrumental, live, valence, tempo, timsig):
        cmd = "UPDATE audio_features SET danceability ='"+str(dance)+"', energy = '"+str(energy)+"', key ='"+str(key)+"',loudness ='"+str(loudness)+"',mode='"+str(mode)+"', speechiness ='"+str(speech)+"',acousticness='"+str(acoustic)+"',instrumentalness='"+str(instrumental)+"',liveness='"+str(live)+"',valence='"+str(valence)+"',tempo='"+str(tempo)+"',t_sig='"+str(timsig)+"'WHERE id= "+str(t_id)
        self.cursor.execute(cmd)
    
    # For Step 12
    def add_dt_col(self):
        cmd = "ALTER TABLE lastfm ADD COLUMN year INTEGER NULL, ADD COLUMN month INTEGER NULL, ADD COLUMN day VARCHAR(10) NULL, ADD COLUMN hour INTEGER NULL"
        self.cursor.execute(cmd)
    def add_datetime(self,t_id, year, month,day,hour):
        cmd = "UPDATE lastfm SET year = '"+str(year)+"',month='"+str(month)+"',day='"+str(day)+"',hour='"+str(hour)+"'WHERE id="+str(t_id)
        self.cursor.execute(cmd)
        
    # For Step 13
    def get_tracks(self, day):
        return pd.read_sql("SELECT * FROM lastfm WHERE day='"+str(day)+"'",self.connection)
    
    def get_af(self):
        return pd.read_sql('SELECT * FROM audio_features', self.connection)

#Connect to database
d = DatabaseConnection()

In [ ]:
## STEP FOUR - Load tokens

#Last.fm api tokens
API_KEY = ''
API_SECRET = ''
username = ""
pass_hash = pylast.md5(getpass.getpass())
network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET, username=username, password_hash=pass_hash)
user = network.get_user(username)

#Spotify api tokens (Timelist)
cli_id = ''
cli_se = ''
client_credentials_manager = SpotifyClientCredentials(client_id= cli_id, client_secret= cli_se)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#Spotify api tokens (Build_playlist)
cli_id2 = ''
cli_se2 = ''
#Enter Spotify developer account user name
sp_username = ""
redirect_url = 'https://example.com/callback'#http://localhost:8888/callback/'
scope = 'playlist-modify-public'
token = util.prompt_for_user_token(sp_username, scope, cli_id2, cli_se2, redirect_uri=redirect_url)
sp2 = spotipy.Spotify(auth=token)

In [ ]:
## STEP FIVE - Get total scrobble history
#            - NOTE: takes about 10-30 minutes to complete

start_time = 1259125200 #epoch time for date when last.fm account was created (Nov 25, 2009 at midnight EST)
a = user.get_recent_tracks(limit=None, cacheable=True, time_from=start_time, time_to=None)
a = list(reversed(a))
print(len(a))

In [ ]:
## STEP SIX - Create table and write scrobble history to table

# make table lastFM to store scrobble history
d.create_table()

# add scrobbles individually to lastFM table in chronological order
count=0
for i in a:
    if int(i.timestamp)>0:
        artist = str(i.track.get_artist())
        if "'" not in artist:
            artist=artist
        else:
            artist = artist.replace("'","''")
        track = str(i.track.get_name())
        if "'" not in track:
            track=track
        else:
            track = track.replace("'","''")
        album = str(i.album)
        if "'" not in album:
            album=album
        else:
            album = album.replace("'","''")
        times = (i.timestamp)
        d.insert_new_record(artist, track, album, times)
        count +=1
print("Tracks added to db:",count)


## STEP SEVEN - Remove rows with duplicate timestamp, leaving the most recent track
#             - Remove rows with less than 30 second playtime

iterate = d.iterate()
num_deleted=0
for i in range (0,len(iterate)):
    if (i+1<len(iterate)):
        if iterate[i+1][4]-iterate[i][4]<=30:
            d.remove_dup(iterate[i][0])
            num_deleted +=1

print("Number of deleted entries: ",num_deleted)

In [ ]:
## STEP EIGHT - Make new table for audio features, and copy all unique tracks into table

d.audio_feature_table()


## STEP NINE - Iterate through audio_features table to see what's in Spotify. Remove things not in Spotify db

#COUNT HOW MANY UNIQUE TRACKS FROM audio_features table NATIVELY HAVE SPOTIFY FEATURES
iterate = d.iter_features()
num_match = 0
for i in range (0,len(iterate)):
    artist = iterate[i][0]
    track = iterate[i][1]
    t_id =iterate[i][3]
    results = (sp.search(q='artist:' + artist + ' track:' + track, type='track', limit=50))
    if results['tracks']['total']==0:
        pass
    for res in range(0, len(results['tracks']['items'])):
        if (artist.lower() == results['tracks']['items'][res]['artists'][0]['name'].lower()) and (track.lower() == results['tracks']['items'][res]['name'].lower()):
            ident = (((results)['tracks']['items'][res]['id']))
            print(i, artist, track, ident)
            d.add_spotID(t_id,ident)
            num_match +=1
            break
print(num_match)


## STEP TEN - Remove rows in audio_features where there is no Spotify ID

d.rem_spot_null()

In [ ]:
## STEP ELEVEN - Add audio features, where available, to the audio_features table
#              - Takes a LONG TIME to run, keep as independent cell

iterate = d.iter_features()
for i in range (0, len(iterate)):
    artist = iterate[i][0]
    track = iterate[i][1]
    t_id = iterate[i][3]
    spotid = iterate[i][4]
    if(sp.audio_features(spotid)[0]!=None):
        print(i," ",artist," - ",track, len(sp.audio_features(spotid)[0]), iterate[i][5])
        if (len(sp.audio_features(spotid)[0]))==18 and iterate[i][5]==None:
            dance = sp.audio_features(spotid)[0]['danceability']
            energy = sp.audio_features(spotid)[0]['energy']
            key = sp.audio_features(spotid)[0]['key']
            loudness = sp.audio_features(spotid)[0]['loudness']
            mode = sp.audio_features(spotid)[0]['mode']
            speech = sp.audio_features(spotid)[0]['speechiness']
            acoustic = sp.audio_features(spotid)[0]['acousticness']
            instrumental = sp.audio_features(spotid)[0]['instrumentalness']
            live = sp.audio_features(spotid)[0]['liveness']
            valence = sp.audio_features(spotid)[0]['valence']
            tempo = sp.audio_features(spotid)[0]['tempo']
            timsig = sp.audio_features(spotid)[0]['time_signature']
            d.feature_track(t_id, dance, energy, key, loudness, mode, speech, 
                            acoustic, instrumental, live, valence, tempo, timsig)

In [ ]:
## STEP TWELVE - Add columns to lastfm table for year, month, day, and hour
#              - Convert UST timestamp to EST -> Year (y), Month (m), Day of week (dw), and Time of day (hour)

#add datetime columns to lastfm table
d.add_dt_col()

#convert timestamps to proper time zone to enter into lastfm table
iterate = d.iterate()
for i in range (0,len(iterate)):
    t_id = iterate[i][0]
    print(iterate[i][4])
    year = (dt.datetime.fromtimestamp(iterate[i][4], gettz("US/Eastern")).year)
    month = (dt.datetime.fromtimestamp(iterate[i][4], gettz("US/Eastern")).month)
    day= (calendar.day_name[dt.datetime.fromtimestamp(iterate[i][4], gettz("US/Eastern")).weekday()])
    hour = (dt.datetime.fromtimestamp(iterate[i][4], gettz("US/Eastern")).hour)
    d.add_datetime(t_id,year,month,day,hour)

In [ ]:
## STEP THIRTEEN - Grab all tracks played on chosen day and feed into dataframe
#                - Inner merge with audio_features dataframe to append audio features to selected tracks
#                - Assign a 0/1 status to dataframe based on if track played during timeframe
#                - NEED TO DO: create selection filter based on "time" variable

day = "Wednesday"
time = 21
day_tracks = d.get_tracks(day)
af = d.get_af()
day_af = pd.merge(day_tracks,af, on=['artist','track'], how='inner')
print(len(day_af), type(day_af))
day_af['status']=''
for i in range(0,len(day_af)):
    h=int(day_af.loc[i,['hour']])
    if 12>h and h>=7:
        day_af.loc[i,['status']]=1
    else:
        day_af.loc[i,['status']]=0
print(day_af['status'].value_counts())


## STEP THIRTEEN PART TWO - Drop columns (id_x, artist, track, album, timestamp, year, 
#                                         month, day, hour, t_played, id_y, and spotid)
#                         - Perform feature selection on remaining columns

day_af_trim=day_af.drop(columns=['id_x', 'artist', 'track', 'album', 'timestamp', 'year', 
                                 'month', 'day', 'hour', 't_played', 'id_y', 'spot_id'])
print(len(day_af_trim))
array = day_af_trim.values
X = array[:,0:12]
Y = array[:,12]
model = LogisticRegression(solver='lbfgs', max_iter=500)
rfe = RFE(model, 1)
fit = rfe.fit(X,Y)
print("Feature ranking: ",fit.ranking_)
max_ft=[]
tar_ft=[]
min_ft=[]
for i in range (0, len(fit.ranking_)):
    if fit.ranking_[i] <5:
        print(day_af_trim.columns.values[i], fit.ranking_[i])
        ones = day_af_trim['status']==1
        firsts = day_af_trim[day_af_trim.columns.values[i]]
        k=day_af_trim[ones & firsts][day_af_trim.columns.values[i]]
        max_ft.append('max_'+day_af_trim.columns.values[i]+' = '+str(k.quantile([0.75]).values[0]))
        tar_ft.append('target_'+day_af_trim.columns.values[i]+' = '+ str(k.quantile([0.50]).values[0]))
        min_ft.append('min_'+day_af_trim.columns.values[i]+' = '+ str(k.quantile([0.25]).values[0]))

In [ ]:
## STEP FOURTEEN PART ONE - Genre seed selection

w=widgets.Dropdown(
    options=["acoustic","afrobeat","alt-rock","alternative","ambient","anime","black-metal","bluegrass","blues",
       "bossanova","brazil","breakbeat","british","cantopop","chicago-house","children","chill","classical",
       "club","comedy","country","dance","dancehall","death-metal","deep-house","detroit-techno","disco",
       "disney","drum-and-bass","dub","dubstep","edm","electro","electronic","emo","folk","forro","french",
       "funk","garage","german","gospel","goth","grindcore","groove","grunge","guitar","happy","hard-rock",
       "hardcore","hardstyle","heavy-metal","hip-hop","holidays","honky-tonk","house","idm","indian","indie",
       "indie-pop","industrial","iranian","j-dance","j-idol","j-pop","j-rock","jazz","k-pop","kids","latin",
       "latino","malay","mandopop","metal","metal-misc","metalcore","minimal-techno","movies","mpb","new-age",
       "new-release","opera","pagode","party","philippines-opm","piano","pop","pop-film","post-dubstep",
       "power-pop","progressive-house","psych-rock","punk","punk-rock","r-n-b","rainy-day","reggae","reggaeton",
       "road-trip","rock","rock-n-roll","rockabilly","romance","sad","salsa","samba","sertanejo","show-tunes",
       "singer-songwriter","ska","sleep","songwriter","soul","soundtracks","spanish","study","summer","swedish",
       "synth-pop","tango","techno","trance","trip-hop","turkish","work-out","world-music"],
    value=None,
    disabled=False,
)
print("Choose a genre:")
display(w)

In [ ]:
## STEP FOURTEEN PART TWO - Recommendation engine

if(w.value!=None):
    print(w.value)
    query = "rec=sp.recommendations(seed_genres=['"+w.value+"'], country='US', limit=10"
    for i in range(0,len(max_ft)):
        query=query+", "+max_ft[i]+", "+tar_ft[i]+", "+min_ft[i]
    query=query+")"
    exec(query)
    print("Spotify recommends the following "+str(len(rec['tracks']))+" tracks:")
    print()
    for i in rec['tracks']:
        print((i['artists'][0]['name']),"-",i['name']," ",i['id'])
        print(i['external_urls']['spotify'])
else:
    print("Please select a genre first before executing this cell.")

In [ ]:
## STEP FOURTEEN PART THREE - Create playlist 

playname = "playlist_name"
playlist = sp2.user_playlist_create(user=sp_username, name=playname, public=True)

playlists = sp2.user_playlists(sp_username)
for item in playlists['items']:
    if item['name']==playname:
        play_id = item['id']

for i in rec['tracks']:
    print(i['id'])
    sp2.user_playlist_add_tracks(user=sp_username, playlist_id=play_id, tracks=[i['id']])